In [ ]:
"""
# import libraries
#!pip install tensorflow
#!pip install --upgrade tensorflow
#!pip install --upgrade keras
#!pip show tensorflow
#!pip install --user tensorflow
#!pip install tensorflow==2.13.1
#!pip install --upgrade scikit-learn
#!pip install keras==2.12.1
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.losses import categorical_crossentropy
#from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
# assign 'date' with 'precipitation' series
data = prcp.values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data)

# Prepare data for LSTM
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        label = data[i+seq_length:i+seq_length+1]
        sequences.append((seq, label))
    return np.array([s[0] for s in sequences]), np.array([s[1] for s in sequences])

seq_length = 30  # Adjust as needed
X, y = create_sequences(data_normalized, seq_length)

# Assuming 'X' and 'y' are your input features and target variable
# Assuming 'seq_length' is your sequence length

# Function to create an LSTM model
def create_lstm_model(units=50, optimizer='adam', dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(units, activation='relu', input_shape=(seq_length, 1)))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Normalize the data
#scaler = MinMaxScaler()
#X_normalized = scaler.fit_transform(X.reshape(-1, 1))

# Reshape X for LSTM input
X_reshaped = X.reshape((X.shape[0], seq_length, 1))

# Create KerasRegressor wrapper for scikit-learn
lstm_model = KerasRegressor(build_fn=create_lstm_model, epochs=50, batch_size=32, verbose=0)

# Define hyperparameters to search over
param_grid = {
    'units': [50, 100],
    'optimizer': ['adam', 'rmsprop'],
    'dropout_rate': [0.2, 0.3]
}

# Use TimeSeriesSplit for time series data
tscv = TimeSeriesSplit(n_splits=5)

# Create GridSearchCV
grid = GridSearchCV(estimator=lstm_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=tscv, verbose=1)

# Fit the model
grid_result = grid.fit(X_reshaped, y)

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
"""